<a href="https://colab.research.google.com/github/technologyhamed/Pandas/blob/main/03_12_Performance_Eval_and_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# High-Performance Pandas: eval and query

As we've already seen in previous chapters, the power of the PyData stack is built upon the ability of NumPy and Pandas to push basic operations into lower-level compiled code via an intuitive higher-level syntax: examples are vectorized/broadcasted operations in NumPy, and grouping-type operations in Pandas.
While these abstractions are efficient and effective for many common use cases, they often rely on the creation of temporary intermediate objects, which can cause undue overhead in computational time and memory use.

To address this, Pandas includes some methods that allow you to directly access C-speed operations without costly allocation of intermediate arrays: `eval` and `query`, which rely on the [NumExpr package](https://github.com/pydata/numexpr).
In this chapter I will walk you through their use and give some rules of thumb about when you might think about using them.

## Motivating query and eval: Compound Expressions

We've seen previously that NumPy and Pandas support fast vectorized operations; for example, when adding the elements of two arrays:

In [51]:
import numpy as np
import pandas as pd

class display_dataFarme(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        # Assume 'args' contains DataFrame objects or their names as strings
        # If it's a string, eval it to get the DataFrame
        # Otherwise, assume it's already a DataFrame
        return '\n'.join(self.template.format(a, (a._repr_html_() if not isinstance(a, str) else eval(a)._repr_html_())) # Modified this line
                         for a in self.args)

    def __repr__(self):
        # Similar to _repr_html_, handle both strings and DataFrames
        return '\n\n'.join(f"{a}:\n{(eval(a) if isinstance(a, str) else a)}"
                           for a in self.args)

class display_dataFarme(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        # Assume 'args' contains DataFrame objects or their names as strings
        # If it's a string, eval it to get the DataFrame
        # Otherwise, assume it's already a DataFrame
        return '\n'.join(self.template.format(a, (eval(a) if isinstance(a, str) else a)._repr_html_())
                         for a in self.args)

    def __repr__(self):
        # Modified to use f-strings and avoid direct addition with the DataFrame representation
        return '\n\n'.join(f"{a}:\n{repr(eval(a) if isinstance(a, str) else a)}"
                           for a in self.args)

class display_dataFarme(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        # Assume 'args' contains DataFrame objects or their names as strings
        # If it's a string, eval it to get the DataFrame
        # Otherwise, assume it's already a DataFrame
        return '\n'.join(self.template.format(a, (eval(a) if isinstance(a, str) else a)._repr_html_())
                         for a in self.args)

    def __repr__(self):
        # Similar to _repr_html_, handle both strings and DataFrames
        return '\n\n'.join(f"{a}:\n{(eval(a) if isinstance(a, str) else a)}"
                           for a in self.args)

In [1]:
import numpy as np
rng = np.random.default_rng(42)
x = rng.random(1000000)
y = rng.random(1000000)
%timeit x + y

2.64 ms ± 547 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As discussed in [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb), this is much faster than doing the addition via a Python loop or comprehension:

In [4]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)),dtype=x.dtype, count=len(x))

205 ms ± 7.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


But this abstraction can become less efficient when computing compound expressions.
For example, consider the following expression:

In [6]:
mask = (x > 0.5) & (y < 0.5)
mask

array([False, False,  True, ...,  True,  True,  True])

Because NumPy evaluates each subexpression, this is roughly equivalent to the following:

In [7]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

In other words, *every intermediate step is explicitly allocated in memory*. If the `x` and `y` arrays are very large, this can lead to significant memory and computational overhead.
The NumExpr library gives you the ability to compute this type of compound expression element by element, without the need to allocate full intermediate arrays.
The [NumExpr documentation](https://github.com/pydata/numexpr) has more details, but for the time being it is sufficient to say that the library accepts a *string* giving the NumPy-style expression you'd like to compute:

In [8]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.all(mask == mask_numexpr)

True

The benefit here is that NumExpr evaluates the expression in a way that avoids temporary arrays where possible, and thus can be much more efficient than NumPy, especially for long sequences of computations on large arrays.
The Pandas `eval` and `query` tools that we will discuss here are conceptually similar, and are essentially Pandas-specific wrappers of NumExpr functionality.

## pandas.eval for Efficient Operations

The `eval` function in Pandas uses string expressions to efficiently compute operations on `DataFrame` objects.
For example, consider the following data:

In [ ]:
import pandas as pd
nrows, ncols = 100000, 100
df1, df2, df3, df4 = (pd.DataFrame(rng.random((nrows, ncols)))
                      for i in range(4))
display('df1', 'df2', 'df3', 'df4')

To compute the sum of all four ``DataFrame``s using the typical Pandas approach, we can just write the sum:

In [12]:
%timeit df1 + df2 + df3 + df4

161 ms ± 62.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The same result can be computed via ``pd.eval`` by constructing the expression as a string:

In [13]:
%timeit pd.eval('df1 + df2 + df3 + df4')

109 ms ± 27.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The `eval` version of this expression is about 50% faster (and uses much less memory), while giving the same result:

In [15]:
c=np.allclose(df1 + df2 + df3 + df4,
            pd.eval('df1 + df2 + df3 + df4'))
%timeit c

14.3 ns ± 4 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


`pd.eval` supports a wide range of operations.
To demonstrate these, we'll use the following integer data:

In [17]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.integers(0, 1000, (100, 3)))
                           for i in range(5))
display('df1', 'df2', 'df3', 'df4', 'df5')

,0,1,2
0,376,963,992
1,816,578,442
2,202,167,501
3,556,336,744
4,202,777,622
...,...,...,...
95,769,576,747
96,279,289,133
97,721,605,116
98,138,986,678


#### Arithmetic operators
`pd.eval` supports all arithmetic operators. For example:

In [18]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)
%timeit result1
%timeit result2


12.7 ns ± 0.294 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
13.7 ns ± 2.62 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


#### Comparison operators
`pd.eval` supports all comparison operators, including chained expressions:

##np.allclose()
np.allclose() is a function in NumPy that checks whether two arrays (or numerical values) are element-wise equal within a tolerance. It's often used for comparing floating-point numbers because floating-point arithmetic can introduce small errors.

In [19]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### Bitwise operators
`pd.eval` supports the `&` and `|` bitwise operators:

In [20]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In addition, it supports the use of the literal `and` and `or` in Boolean expressions:

In [29]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

False

#### Object attributes and indices

`pd.eval` supports access to object attributes via the `obj.attr` syntax and indexes via the `obj[index]` syntax:

df2.T[0]: This transposes the df2 DataFrame (df2.T), then accesses the first column of the transposed DataFrame (which was originally the first row of df2).

df3.iloc[1]: This accesses the second row (index 1) of df3 using the .iloc[] indexer.

In [21]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### Other operations

Other operations, such as function calls, conditional statements, loops, and other more involved constructs are currently *not* implemented in `pd.eval`.
If you'd like to execute these more complicated types of expressions, you can use the NumExpr library itself.

## DataFrame.eval for Column-Wise Operations

Just as Pandas has a top-level `pd.eval` function, `DataFrame` objects have an `eval` method that works in similar ways.
The benefit of the `eval` method is that columns can be referred to by name.
We'll use this labeled array as an example:

In [34]:
df = pd.DataFrame(rng.random((1000, 3)), columns=['A', 'B', 'C'])
df.head()
display_dataFarme(df)

,A,B,C
0,0.045598,0.758163,0.705888
1,0.502500,0.224937,0.143135
2,0.968299,0.418945,0.187373
3,0.786622,0.394693,0.581836
4,0.327628,0.735279,0.598284
...,...,...,...
995,0.111151,0.242489,0.121566
996,0.234786,0.899766,0.078242
997,0.570301,0.155584,0.871099
998,0.697893,0.471439,0.733376


Using `pd.eval` as in the previous section, we can compute expressions with the three columns like this:

In [35]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

The `DataFrame.eval` method allows much more succinct evaluation of expressions with the columns:

In [37]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

Notice here that we treat *column names as variables* within the evaluated expression, and the result is what we would wish.

### Assignment in DataFrame.eval

In addition to the options just discussed, `DataFrame.eval`  also allows assignment to any column.
Let's use the `DataFrame` from before, which has columns `'A'`, `'B'`, and `'C'`:

In [38]:
df.head()

,A,B,C
0,0.045598,0.758163,0.705888
1,0.502500,0.224937,0.143135
2,0.968299,0.418945,0.187373
3,0.786622,0.394693,0.581836
4,0.327628,0.735279,0.598284


We can use `df.eval` to create a new column `'D'` and assign to it a value computed from the other columns:

In [39]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.045598,0.758163,0.705888,1.138652
1,0.502500,0.224937,0.143135,5.082169
2,0.968299,0.418945,0.187373,7.403644
3,0.786622,0.394693,0.581836,2.030324
4,0.327628,0.735279,0.598284,1.776593


#In the same way, any existing column can be modified:

In [44]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.045598,0.758163,0.705888,-1.009459
1,0.502500,0.224937,0.143135,1.939165
2,0.968299,0.418945,0.187373,2.931875
3,0.786622,0.394693,0.581836,0.673608
4,0.327628,0.735279,0.598284,-0.681367


### Local Variables in DataFrame.eval

The `DataFrame.eval` method supports an additional syntax that lets it work with local Python variables.
Consider the following:

In [40]:
column_mean = df.mean(1) #axis=1 Row | axis=0 columns
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

 **The `@` character here marks a *variable name* rather than a *column name*, and lets you efficiently evaluate expressions involving the two "namespaces": the namespace of columns, and the namespace of Python objects.
Notice that this `@` character is only supported by the `DataFrame.eval` *method*, not by the `pandas.eval` *function*, because the `pandas.eval` function only has access to the one (Python) namespace.**

## The DataFrame.query Method

The `DataFrame` has another method based on evaluated strings, called `query`.
Consider the following:

In [45]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

As with the example used in our discussion of `DataFrame.eval`, this is an expression involving columns of the `DataFrame`.
However, it cannot be expressed using the `DataFrame.eval` syntax!
Instead, for this type of filtering operation, you can use the `query` method:

In [43]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In addition to being a more efficient computation, compared to the masking expression this is much easier to read and understand.
Note that the `query` method also accepts the `@` flag to mark local variables:

In [46]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

## Performance: When to Use These Functions

When considering whether to use `eval` and `query`, there are two considerations: *computation time* and *memory use*.
Memory use is the most predictable aspect. As already mentioned, every compound expression involving NumPy arrays or Pandas ``DataFrame``s will result in implicit creation of temporary arrays. For example, this:

In [53]:
x = df[(df.A < 0.5) & (df.B < 0.5)]
display_dataFarme(x)

,A,B,C,D
5,0.033389,0.163019,0.806914,-0.160650
7,0.293520,0.363281,0.163106,-0.427707
8,0.188097,0.262411,0.549008,-0.135362
10,0.246889,0.145728,0.533356,0.189670
22,0.345104,0.394241,0.753509,-0.065212
...,...,...,...,...
977,0.048267,0.157808,0.124810,-0.877660
980,0.003419,0.307212,0.785248,-0.386875
984,0.289053,0.092584,0.306070,0.641911
987,0.476049,0.081260,0.699442,0.564434


is roughly equivalent to this:

In [54]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

If the size of the temporary ``DataFrame``s is significant compared to your available system memory (typically several gigabytes), then it's a good idea to use an `eval` or `query` expression.
You can check the approximate size of your array in bytes using this:

In [55]:
df.values.nbytes

32000

On the performance side, `eval` can be faster even when you are not maxing out your system memory.
The issue is how your temporary objects compare to the size of the L1 or L2 CPU cache on your system (typically a few megabytes); if they are much bigger, then `eval` can avoid some potentially slow movement of values between the different memory caches.
In practice, I find that the difference in computation time between the traditional methods and the `eval`/`query` method is usually not significant—if anything, the traditional method is faster for smaller arrays!
The benefit of `eval`/`query` is mainly in the saved memory, and the sometimes cleaner syntax they offer.

We've covered most of the details of `eval` and `query` here; for more information on these, you can refer to the Pandas documentation.
In particular, different parsers and engines can be specified for running these queries; for details on this, see the discussion within the ["Enhancing Performance" section](https://pandas.pydata.org/pandas-docs/dev/user_guide/enhancingperf.html) of the documentation.